## Simulate non-cyanogenic evolution via drift alone

There is a finding that across a cline from less to more urban the frequency of a non-cyanogenic phenotype increases
The phenotype is genetically controlled by two loci, both of which have a segregating knock-out allele
If any individual is homozygous for either knockout they become non-cyanogenic

In this extremely simple simulation I create a 'population' which is represented by 2 lists of alleles (A/a and B/b)
To simulate evolution I randomly sample with replacement from the lists to create new list that represent the next generation.

By repeating this process with variable starting frequencies, population sizes and numbers of generations (functionally equivalent to steps in a strict stepping stone) we can look at the change in the frequency of cyanogenic and non-cyanogenic phenotypes

## Extension to previous simulations — Spatial structure

To build off of the previous single-population stepping stone model, I have added a few functions that allow us to add spatial structure to our simulations, effectively simulating the trajectory of multiple populations simultaneously. Once migration is added, this will be analogous to a metapopulation.

#### Major changes to script

1. Since there will be multiple populations, they are now stored as entries in a dictionary. Each key in the dictionarry corresponds to a population and the value is a list containing the frequencies of alleles phenotype, updated each generation. 
2. The alleles that correspond to each population are stored in a separate dictionary with keys matching those from the population dictionary. As such, allele frequencies for populations are calculated by sampling the allele lists in the 'alleles' dictionary with the key matching the population.
3. Every generation, each population has some probability (p) of generating a new population with N alleles sampled from the population that created it. This is not the most realistic scenario but can easily be modified later. 
4. Simulations are now stored separately in a dictionary called 'sim' where each key corresponds to a simulation. 
5. These changes allow us to model clines in two ways:
    - Clines across 'time': looking at phenotype frequencies **within** populations
    - Clines across 'space': looking at phenotype frequencies **across** created populations

## Extension to previous simulations — Logistic population growth

To build off of the simulations that included spatial structure, I have incorporated logistic population growth into the simulations. 

**Major changes to script**

1. Newly created populations form as a bottleneck from the population that created it. The proportion of alleles sampled is some fraction (bot) multiplied by the population size of the source population. 
2. Every generation, every population grows in size according to an approximate logistic growth curve. The logistic growth function incorporates a fractional constant that, when multiplied by the current population size, modifies the per capita growth rate of the population size. The carrying capacity is thus altered by changing this fractional constant. 
3. The size of every population as an integer in a list within the alleles dictionary under the key corresponding to the population. This integer is called as needed by the function that require population size as an argument (e.g. 'bottle', 'pop_growth').

In [10]:
# Modules used throughout script
import random
from collections import OrderedDict
import csv
import time
from datetime import datetime
import os
import itertools
import math
import numpy as np


# Randomly sample 'N' alleles from lists containing alleles for locus A 
def sample_population_A(locus_A, N):
    new_locus_A = [random.choice(locus_A) for _ in range(N)]
    return new_locus_A

# Randomly sample 'N' alleles from lists containing alleles for locus B
def sample_population_B(locus_B, N):
    new_locus_B = [random.choice(locus_B) for _ in range(N)]
    return new_locus_B

# From list containing alleles, calculate the frequency of 'A' or 'B' allele. 
def allele_freq(locus):
    p = sum(1*i.isupper() for i in locus)/float(len(locus))
    return p

# Function for logistic population growth. Takes current population size (from alleles dictionary) as input and return new population
# size. 
def pop_growth(r, Akey, Avalue, alleles):
    size = Avalue['S'][0] # Retrieve size of population. 'Akey' allows indexing of alleles dictionary in cline function
    R = r - (0.01*size) # Calculate modified growth rate. Modifier constant currently needs to be changed manually based on desired carrying capacity
    new_size = [int(math.ceil(R*size))] # Calculate new population size
    return new_size # Update population size in alleles dictionary

# Simple bottleneck function. Uses desired bottleneck proportion and returns an integer of number of alleles to sample based on population size.
# Only used when creating populations
def bottle(bot, Akey, Avalue):
    return int(math.ceil(bot*Avalue['S'][0]))

# Create new population as empty list and add to 'pops' dictionary. Also create two lists of alleles
# sampled from pool of alleles from population that generated the new one. Alleles added to 'alleles'
# dictionary
def create_population(p, Akey, Avalue, pops, alleles, bot): #'Akey' and 'Avalue' arguments local variables defined in 'cline' function
    if not alleles['1']['A']: #If there are no alleles for first population, pass. Only valid for first iteration when the populations have yet to be initialized
        #print 'There are no populations from which to sample!!'
        pass
    else:
        prob_list = (['1'] * int(10*p) ) + (['0'] * int(round(10*(1-p)))) #List with 0's and 1's based on argument 'p'
        create = [random.choice(prob_list) for _ in range(1)] #Randomly select an element from 'prob_list'
        if create[0] == '1': #If a '1' is sampled, create population
            global pop_counter #global variable that tracks the number of populations created
            pop_counter += 1 #Increment 'pop_counter' by 1 if population is being created. 
            #Add two lists to 'alleles' dictionary ('A' and 'B'). Naming: 'Pop.number_Population that created the new one'
            alleles['{0}.{1}'.format(pop_counter,Akey)] = {'A':sample_population_A(alleles[Akey].items()[0][1], bottle(bot, Akey, Avalue)),'B':sample_population_B(alleles[Akey].items()[2][1], bottle(bot, Akey, Avalue)), 'S':[bottle(bot, Akey, Avalue)]}
            pops['{0}.{1}'.format(pop_counter,Akey)] = [] #Empty list for new population. Naming same as alleles.
            
# Given the frequencies of 'A' and 'B' alleles, return the frequency of the 'acyanogenic' phenotype (i.e. recessive
# at either the A locus, B locus, or both) 
def phenotype(pA, pB):
    qA = 1-pA
    qB = 1-pB
    mut= qA**2 + qB**2 - (qA**2 * qB**2)
    WT = 1-mut
    return mut # Frequency of acyanogenic phenotype

# Cline function. Every generation, 'N' alleles are sampled for pool of alleles sampled the previous generation.
# Ever generation, every population has some probability (p) of generating a new population, with alleles
# sampled from the population that created it.
def cline(locus_A,locus_B, steps, N, p, pops, alleles, bot):
        for i in range(steps):
            for Akey, Avalue in alleles.items():
                if Akey in pops.keys():
                    if 'A' and 'B' in Avalue.keys():
                        if not Avalue['A'] and not Avalue['B']: 
                            #If allele lists are empty, sample from list of initial allele frequencies. Only used for first generation
                            Avalue['S'] = [N]
                            Avalue['A'] = (sample_population_A(locus_A, N))
                            Avalue['B'] = (sample_population_B(locus_B, N))
                        else:
                            #If allele lists are not empty, sample from previously sampled set of alleles. 
                            Avalue['S'] = pop_growth(r, Akey, Avalue, alleles)
                            Avalue['A'] = (sample_population_A(Avalue['A'], Avalue['S'][0]))
                            Avalue['B'] = (sample_population_B(Avalue['B'], Avalue['S'][0]))
                    create_population(p, Akey, Avalue, pops, alleles, bot) #Create population. Alleles will be sampled (see above). Population is currently empty list
            for Pkey in pops.keys():
                #Calculate allele and phetype frequencies for every population, including newly created ones. 
                pA = allele_freq(alleles[Pkey].items()[0][1]) 
                pB = allele_freq(alleles[Pkey].items()[2][1])
                pops[Pkey].append([alleles[Pkey].items()[1][1][0], i, pA, pB, phenotype(pA, pB)])
        return pops

        
# Using the functions defined above, 'simulate' performs 'sims' iterations of the cline function -- simulating the effects 
# of drift in a stepping stone model -- each time storing the results.
def simulate(pA, pB, steps, N, sims):
    qA = 1-pA # Frequency of 'a' allele
    qB = 1-pB
    # Make the two lists based on the allele frequency to represent the initial population
    locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) ) # [A,A,A,A,a,a,a,a,....]
    locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) ) 
    ####### sims simulations #####################
    # We will simulate 'steps' iterations of resampling this population to simulate drift
    # We will then repeat that simulation of 'steps' iterations 1000 times to get a mean
    ##############################################
    for s in range(sims):
        pops = OrderedDict({'1':[]}) # Re-initialize dictionary to store populations
        alleles = OrderedDict({'1':{'A':[],'B':[],'S':[]}}) # Re-initialize dictionary to store allele lists
        global pop_counter # Reset population counter
        pop_counter = 1
        # reset the population for each iteration. I don't actually think this is necessary
        locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) )  # Re-initialize initial allele lists.
        locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) ) 
        cline(locus_A,locus_B, steps, N, p, pops, alleles, bot) # Run cline function
        global sim
        sim[s] = pops # Append results to global 'sim' dictionary
        


In [31]:
N = 100 # Starting population size (i.e. sample 10 alleles)
pA = 0.5 # Initial frequency of 'A' allele
pB = 0.5 # Initial frequency of 'B' allele
qA = 1 - pA # 'a' allele
qB = 1 - pB # 'b' allele
steps = 5 # Number of generations
sims = 1 # Number of simulations
bot = 0.1 # Bottleneck proportion
pop_counter = 1 # Count of number of populations being created. Modified by 'create_population'. Used in naming populations
p = 1 # Probability of creating a population. 
r = 2 # Growth rate
sim = {} # Global dictionary used for storing results of each iteration
simulate(pA, pB, steps, N, sims) # Simulate function. 
sim

{0: OrderedDict([('1',
               [[100, 0, 0.46, 0.46, 0.49816944],
                [100, 1, 0.42, 0.55, 0.47077900000000006],
                [100, 2, 0.35, 0.56, 0.534304],
                [100, 3, 0.42, 0.6, 0.4425760000000001],
                [100, 4, 0.47, 0.63, 0.37934479000000004]]),
              ('2.1',
               [[10, 0, 0.8, 0.3, 0.5104],
                [19,
                 1,
                 0.7894736842105263,
                 0.42105263157894735,
                 0.3646457593173779],
                [35, 2, 0.8, 0.3142857142857143, 0.491395918367347],
                [58,
                 3,
                 0.7758620689655172,
                 0.20689655172413793,
                 0.6476506508728498],
                [83,
                 4,
                 0.6987951807228916,
                 0.27710843373493976,
                 0.5658865386324982]]),
              ('3.1',
               [[10, 1, 0.4, 0.4, 0.5904],
                [19,
                 2

In [19]:
for value in sim.values():
    for key in value.keys():
        split = key.split('.')
        print split
        dis = len(split) - 1
        if dis == 0:
            print dis, 'There is no migration!'
        elif dis == 1:
            print dis, 'The migration rate is 0.4'
        elif dis == 2:
            print dis, 'the migration rate is 0.3'
        else:
            print dis, 'The migration rate is 0.2'

['1']
0 There is no migration!
['2', '1']
1 The migration rate is 0.4
['3', '1']
1 The migration rate is 0.4
['4', '3', '1']
2 the migration rate is 0.3
['5', '2', '1']
2 the migration rate is 0.3
['6', '4', '3', '1']
3 The migration rate is 0.2
['7', '1']
1 The migration rate is 0.4
['8', '2', '1']
2 the migration rate is 0.3
['9', '5', '2', '1']
3 The migration rate is 0.2


In [35]:
from itertools import combinations
for value in sim.itervalues():
    for key in itertools.combinations(value.iterkeys(),2):
        print key, abs(len(key[1].split('.')) - len(key[0].split('.')))


('1', '2.1') 1
('1', '3.1') 1
('1', '4.2.1') 2
('1', '5.1') 1
('1', '6.2.1') 2
('1', '7.3.1') 2
('1', '8.4.2.1') 3
('1', '9.1') 1
('1', '10.2.1') 2
('1', '11.3.1') 2
('1', '12.4.2.1') 3
('1', '13.5.1') 2
('1', '14.6.2.1') 3
('1', '15.7.3.1') 3
('1', '16.8.4.2.1') 4
('1', '17.1') 1
('1', '18.2.1') 2
('1', '19.3.1') 2
('1', '20.4.2.1') 3
('1', '21.5.1') 2
('1', '22.6.2.1') 3
('1', '23.7.3.1') 3
('1', '24.8.4.2.1') 4
('1', '25.9.1') 2
('1', '26.10.2.1') 3
('1', '27.11.3.1') 3
('1', '28.12.4.2.1') 4
('1', '29.13.5.1') 3
('1', '30.14.6.2.1') 4
('1', '31.15.7.3.1') 4
('1', '32.16.8.4.2.1') 5
('2.1', '3.1') 0
('2.1', '4.2.1') 1
('2.1', '5.1') 0
('2.1', '6.2.1') 1
('2.1', '7.3.1') 1
('2.1', '8.4.2.1') 2
('2.1', '9.1') 0
('2.1', '10.2.1') 1
('2.1', '11.3.1') 1
('2.1', '12.4.2.1') 2
('2.1', '13.5.1') 1
('2.1', '14.6.2.1') 2
('2.1', '15.7.3.1') 2
('2.1', '16.8.4.2.1') 3
('2.1', '17.1') 0
('2.1', '18.2.1') 1
('2.1', '19.3.1') 1
('2.1', '20.4.2.1') 2
('2.1', '21.5.1') 1
('2.1', '22.6.2.1') 2
('2.1'

In [524]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [564]:
pops = OrderedDict({'1':[]}) # Re-initialize dictionary to store populations
alleles = OrderedDict({'1':{'A':[],'B':[],'S':[]}}) # Re-initialize dictionary to store allele lists
N = 10 # Starting population size (i.e. sample 10 alleles)
pA = 0.5 # Initial frequency of 'A' allele
pB = 0.5 # Initial frequency of 'B' allele
qA = 1 - pA # 'a' allele
qB = 1 - pB # 'b' allele
steps = 5 # Number of generations
sims = 2 # Number of simulations
bot = int(round(N*0.1))
pop_counter = 1 # Count of number of populations being created. Modified by 'create_population'. Used in naming populations
p = 1 # Probability of creating a population. 
locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) ) # [A,A,A,A,a,a,a,a,....]
locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) )
r = 1.5

In [565]:
cline(locus_A,locus_B, steps, N, p, pops, alleles, bot)

ZeroDivisionError: float division by zero

In [590]:
pA = 0.5 # Initial frequency of 'A' allele
pB = 0.5 # Initial frequency of 'B' allele
qA = 1 - pA # 'a' allele
qB = 1 - pB
pop_counter = 1
N = 10
locus_A = (['A'] * int(N*pA) ) + (['a'] * int(round(N*qA)) ) # [A,A,A,A,a,a,a,a,....]
locus_B = (['B'] * int(N*pB) ) + (['b'] * int(round(N*qB)) ) 
r = 2
bot = 0.1
pops = OrderedDict({'1':[]}) # Re-initialize dictionary to store populations
alleles = OrderedDict({'1':{'A':sample_population_A(locus_A, N),'B':sample_population_B(locus_B, N),'S':[N]}})

In [1]:
for i in range (2):
    for Akey, Avalue in alleles.items():
        if Akey in pops.keys():
            Avalue['S'] = pop_growth(r, Akey, Avalue, alleles)
            Avalue['A'] = (sample_population_A(Avalue['A'], Avalue['S'][0]))
            Avalue['B'] = (sample_population_B(Avalue['B'], Avalue['S'][0]))
            create_population(p, Akey, Avalue, pops, alleles, bot)

NameError: name 'alleles' is not defined

In [37]:
COLS, ROWS = 5,5 
rows = [i for i in range(ROWS)]
columns = [i for i in range(COLS)]
test = {c : {i:0 for i in rows} for c in columns}

In [38]:
test

{0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
 1: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
 2: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
 3: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
 4: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}}